In [1]:
print("ok")

ok


In [2]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

In [3]:
%pwd

'd:\\source_code_analysis\\End-to-End_Source_code_Analysis_GenerativeAI\\research'

In [4]:
!mkdir test_repo

In [5]:
repo_path = "test_repo/"

repo = Repo.clone_from(
    "https://github.com/Ahsan-techie/Medical_Chatbot_GenAI.git",
    to_path=repo_path,
)

In [6]:
loader = GenericLoader.from_filesystem(
    repo_path,
    glob="**/*",
    suffixes=[".py"],
    parser=LanguageParser(language = Language.PYTHON, parser_threshold=500),
)

In [7]:
documents = loader.load()

In [8]:
documents

[Document(metadata={'source': 'test_repo\\app.py', 'language': <Language.PYTHON: 'python'>}, page_content='from flask import Flask, render_template, jsonify, request\nfrom src.helper import download_hugging_face_embeddings\nfrom langchain_pinecone import PineconeVectorStore\nfrom langchain_openai import OpenAI\nfrom langchain.chains import create_retrieval_chain\nfrom langchain.chains.combine_documents import create_stuff_documents_chain\nfrom langchain_core.prompts import ChatPromptTemplate\nfrom dotenv import load_dotenv\nfrom src.prompt import *\nimport os\n\napp = Flask(__name__)\n\nload_dotenv()\n\nPINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")\nOPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")\n\nos.environ["PINECONE_API_KEY"] = PINECONE_API_KEY\nos.environ["OPENAI_API_KEY"] = OPENAI_API_KEY\n\nembeddings = download_hugging_face_embeddings()\n\n\nindex_name = "medibot"\n\n# Embed each chunk and upsert the embeddings into your Pinecone index.\ndocsearch = PineconeVectorStor

In [9]:
len(documents)

7

In [10]:
documents_splitter = RecursiveCharacterTextSplitter.from_language(
    language = Language.PYTHON,
    chunk_size=500,
    chunk_overlap=20
)

In [11]:
codes = documents_splitter.split_documents(documents)

In [12]:
codes

[Document(metadata={'source': 'test_repo\\app.py', 'language': <Language.PYTHON: 'python'>}, page_content='from flask import Flask, render_template, jsonify, request\nfrom src.helper import download_hugging_face_embeddings\nfrom langchain_pinecone import PineconeVectorStore\nfrom langchain_openai import OpenAI\nfrom langchain.chains import create_retrieval_chain\nfrom langchain.chains.combine_documents import create_stuff_documents_chain\nfrom langchain_core.prompts import ChatPromptTemplate\nfrom dotenv import load_dotenv\nfrom src.prompt import *\nimport os\n\napp = Flask(__name__)\n\nload_dotenv()'),
 Document(metadata={'source': 'test_repo\\app.py', 'language': <Language.PYTHON: 'python'>}, page_content='load_dotenv()\n\nPINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")\nOPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")\n\nos.environ["PINECONE_API_KEY"] = PINECONE_API_KEY\nos.environ["OPENAI_API_KEY"] = OPENAI_API_KEY\n\nembeddings = download_hugging_face_embeddings()\n\n\ninde

In [13]:
len(codes)

13

In [14]:
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [15]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [16]:
embeddings = OpenAIEmbeddings(disallowed_special=())

C:\Users\Prime Laptop\AppData\Local\Temp\ipykernel_17172\805463212.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(disallowed_special=())


In [37]:
vectorstore = Chroma(
    persist_directory="./chroma_db",  # directory where the vectorstore is persisted
    collection_name="code-snippets",  # collection name used when storing your documents
    embedding_function=embeddings,  # embedding function from OpenAIEmbeddings
)

In [38]:
vectorstore.add_documents(codes)
print("Embeddings computed and stored in Chroma!")

Embeddings computed and stored in Chroma!


In [39]:
llm = ChatOpenAI()

In [40]:
memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history",output_key='answer', return_messages=True)

In [41]:
qa = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    memory=memory,
    return_source_documents=True,
    verbose=True,
)

In [42]:
qa.output_key = 'answer'

In [43]:
question = "what is download_hugging_face_embeddings funtion?"

In [44]:
result = qa.invoke({"question":question})
print(result["answer"])



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
def text_split(extracted_data):
    text_slpitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_slpitter.split_documents(extracted_data)
    return text_chunks


def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    return embeddings

from src.helper import load_pdf_file, text_split, download_hugging_face_embeddings
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from dotenv import load_dotenv
import os


load_dotenv()

PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
o

In [45]:
question = "what is load_pdf_file funtion?"
result = qa.invoke({"question":question})
print(result["answer"])



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

system: The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential. The human asks what the `download_hugging_face_embeddings` function is. The AI explains that it is a function that downloads a specific pre-trained model for generating embeddings from the Hugging Face model hub, specifically the "sentence-transformers/all-MiniLM-L6-v2" model.
Follow Up Input: what is load_pdf_file funtion?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that yo